In [ ]:
from copula_models import GaussianFactorCopulaSSM, StudentFactorCopulaSSM, GroupedStudentFactorCopulaSSM, simulate_copula_model
import particles 
from particles import state_space_models as ssm

# Exemple : Simuler et filtrer le Gaussien
model_g = GaussianFactorCopulaSSM(n_series=10)
_, data_g = simulate_copula_model(model_g, T=200)

fk_g = ssm.Bootstrap(ssm=model_g, data=data_g)
pf_g = particles.SMC(fk=fk_g, N=1000)
pf_g.run()
print("LogLik Gaussien:", pf_g.logLt)

# Exemple : Simuler et filtrer le Grouped Student
model_grp = GroupedStudentFactorCopulaSSM(n_series=10, n_groups=2, nus=[5, 20])
_, data_grp = simulate_copula_model(model_grp, T=200)
# ... etc